# 1 — Black–Scholes Basics

In this notebook we:

1. Recall the assumptions of the Black–Scholes model.
2. Derive and write down the formulas for $d_1$, $d_2$, call and put prices.
3. Interpret $d_1$, $d_2$, $N(d_1)$ and $N(d_2)$.
4. Compute example option prices with the implemented functions.
5. Decompose option value into intrinsic value and time value.
6. Check put–call parity.
7. Apply the formula to one sample quote from the SQL database.


In [1]:
import sys
from pathlib import Path

# Adjust this path if needed
project_root = Path(r"C:\Users\Ruben\Desktop\Projects\OptionPricing")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from option_pricing.black_scholes import black_scholes_call, black_scholes_put
from option_pricing.greeks import d1, d2
from option_pricing.utils import annualize_days
from option_pricing.db import get_connection

print("Project root:", project_root)


Project root: C:\Users\Ruben\Desktop\Projects\OptionPricing


## 1. Assumptions of the Black–Scholes model

The classical Black–Scholes model for a non-dividend-paying stock is built on the following assumptions:

1. The underlying stock price follows a geometric Brownian motion with constant drift and volatility:
   $$
   dS_t = \mu S_t \, dt + \sigma S_t \, dW_t.
   $$
2. The volatility $\sigma$ and the risk-free rate $r$ are constant over the option's lifetime.
3. Trading is continuous, there are no transaction costs or taxes, and assets are perfectly divisible.
4. It is possible to borrow and lend at the same risk-free rate $r$.
5. Markets are frictionless and there are no arbitrage opportunities.
6. The option is **European**: it can be exercised only at maturity $T$.
7. The underlying pays no dividends (we will add dividend yield later if needed).

Under these assumptions, we can derive a closed-form formula for the price of a European call or put option.


## 2. Black–Scholes formulas

For a European call and put option on a non-dividend-paying stock, the Black–Scholes prices are:

- Call:
  $$
  C = S_0 N(d_1) - K e^{-rT} N(d_2),
  $$
- Put:
  $$
  P = K e^{-rT} N(-d_2) - S_0 N(-d_1),
  $$

where

$$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + \frac{\sigma^2}{2}\right) T}{\sigma \sqrt{T}},
\qquad
d_2 = d_1 - \sigma \sqrt{T},
$$

and:

- $S_0$ — current underlying price,
- $K$ — strike price,
- $T$ — time to maturity (in years),
- $r$ — continuously compounded risk-free rate,
- $\sigma$ — volatility of the underlying,
- $N(\cdot)$ — standard normal cumulative distribution function.


In [2]:
S0 = 100.0   # underlying price
K = 100.0    # strike
T = 0.5      # 0.5 years to maturity (6 months)
r = 0.02     # 2% risk-free rate
sigma = 0.25 # 25% volatility

d1_val = d1(S0, K, T, r, sigma)
d2_val = d2(S0, K, T, r, sigma)

d1_val, d2_val


(0.14495689014324226, -0.03181980515339464)

## 3. Interpretation of $d_1$, $d_2$, $N(d_1)$ and $N(d_2)$

The variables $d_1$ and $d_2$ are standardized (normalized) quantities that come from the
lognormal distribution of the stock price under the risk-neutral measure.

- $d_1$ can be seen as a **signal-to-noise ratio**:
  it compares the expected (risk-neutral) growth of the stock to the volatility over the horizon $T$.
- $d_2 = d_1 - \sigma \sqrt{T}$ is $d_1$ shifted down by one volatility unit over the time horizon.

Economic interpretations:

- $N(d_1)$ is closely related to the **delta** of the call option,
  and can be interpreted (under some conditions) as the **risk-neutral probability that the option
  finishes in-the-money, weighted by the hedge ratio**.
- $N(d_2)$ is often interpreted as the **risk-neutral probability that the option finishes in-the-money**:
  $$
  N(d_2) \approx \mathbb{Q}(S_T > K),
  $$
  where $\mathbb{Q}$ is the risk-neutral measure.

Because of these interpretations, $N(d_1)$ and $N(d_2)$ are sometimes called **"probabilities"** in informal explanations, although strictly speaking they are probabilities under the risk-neutral measure, not the real-world measure.


In [3]:
call_price = black_scholes_call(S0, K, T, r, sigma)
put_price = black_scholes_put(S0, K, T, r, sigma)

print(f"Call price: {call_price:.4f}")
print(f"Put price:  {put_price:.4f}")


Call price: 7.5168
Put price:  6.5218


## 4. Intrinsic value and time value

The Black–Scholes price of an option consists of two parts:

1. **Intrinsic value** — the value if we exercise the option immediately.
2. **Time value** — the extra value coming from the possibility that the situation improves before maturity.

For a call option:

- intrinsic value: $\max(S_0 - K, 0)$,
- time value: $C - \max(S_0 - K, 0)$.

For a put option:

- intrinsic value: $\max(K - S_0, 0)$,
- time value: $P - \max(K - S_0, 0)$.

Time value is always non-negative in the Black–Scholes framework.


In [4]:
call_intrinsic = max(S0 - K, 0.0)
put_intrinsic = max(K - S0, 0.0)

call_time_value = call_price - call_intrinsic
put_time_value = put_price - put_intrinsic

print(f"Call intrinsic value: {call_intrinsic:.4f}")
print(f"Call time value:      {call_time_value:.4f}")

print(f"Put intrinsic value:  {put_intrinsic:.4f}")
print(f"Put time value:       {put_time_value:.4f}")


Call intrinsic value: 0.0000
Call time value:      7.5168
Put intrinsic value:  0.0000
Put time value:       6.5218


## 5. Put–call parity

For European options on a non-dividend-paying stock, **put–call parity** states that:

$$
C - P = S_0 - K e^{-rT},
$$

where $C$ is the call price and $P$ is the put price with the same strike $K$ and maturity $T$.

Rearranged:

- Given $C$, we can compute the "fair" $P$:
  $$
  P = C - S_0 + K e^{-rT}.
  $$
- Given $P$, we can compute the "fair" $C$:
  $$
  C = P + S_0 - K e^{-rT}.
  $$

If market prices violate put–call parity, there is an arbitrage opportunity in the idealized frictionless model.


In [5]:
import math

lhs = call_price - put_price
rhs = S0 - K * math.exp(-r * T)

print(f"LHS (C - P)       = {lhs:.10f}")
print(f"RHS (S0 - K e^-rT) = {rhs:.10f}")
print(f"Difference         = {lhs - rhs:.10e}")


LHS (C - P)       = 0.9950166251
RHS (S0 - K e^-rT) = 0.9950166251
Difference         = 0.0000000000e+00


## 6. Pricing a sample option from the SQL database

Now we connect to the SQLite database `data/option_pricing.db`, read one of the sample
option quotes inserted in notebook 0, and apply the Black–Scholes formula.

In notebook 2 we will compute **implied volatility** from the market quote.  
Here we simply assume some volatility (for example $\sigma = 0.25$) and compare
the model price to the mid market price.


In [6]:
import datetime as dt
import math

conn = get_connection()

# Take one sample quote (for example the AAPL call we inserted)
row = conn.execute("""
SELECT *
FROM option_quotes
WHERE underlying_symbol = 'AAPL'
  AND option_type = 'CALL'
LIMIT 1;
""").fetchone()

conn.close()

row_dict = dict(row)
row_dict


{'id': 1,
 'underlying_symbol': 'AAPL',
 'quote_date': '2025-01-03',
 'expiration_date': '2025-03-21',
 'strike': 210.0,
 'option_type': 'CALL',
 'bid': 5.1,
 'ask': 5.4,
 'mid': 5.25,
 'underlying_price': 212.2,
 'risk_free_rate': 0.045,
 'dividend_yield': 0.0,
 'source': 'MANUAL'}

In [7]:
quote_date = dt.date.fromisoformat(row_dict["quote_date"])
expiration_date = dt.date.fromisoformat(row_dict["expiration_date"])
days_to_maturity = (expiration_date - quote_date).days
T_sample = annualize_days(days_to_maturity)

S0_sample = row_dict["underlying_price"]
K_sample = row_dict["strike"]
r_sample = row_dict["risk_free_rate"]

# For now we assume a volatility; in notebook 2 we will back out implied vol
sigma_sample = 0.25

call_model = black_scholes_call(S0_sample, K_sample, T_sample, r_sample, sigma_sample)

print(f"Quote date:         {quote_date}")
print(f"Expiration date:    {expiration_date}")
print(f"Days to maturity:   {days_to_maturity}")
print(f"T (years):          {T_sample:.6f}")
print(f"Underlying price:   {S0_sample}")
print(f"Strike:             {K_sample}")
print(f"Risk-free rate:     {r_sample}")
print(f"Assumed sigma:      {sigma_sample}")
print(f"Market mid price:   {row_dict['mid']}")
print(f"Model call price:   {call_model:.4f}")
print(f"Difference (model - market): {call_model - row_dict['mid']:.4f}")


Quote date:         2025-01-03
Expiration date:    2025-03-21
Days to maturity:   77
T (years):          0.210959
Underlying price:   212.2
Strike:             210.0
Risk-free rate:     0.045
Assumed sigma:      0.25
Market mid price:   5.25
Model call price:   11.8557
Difference (model - market): 6.6057


# Conclusion

In this notebook we:

- Reviewed the assumptions of the Black–Scholes model.
- Wrote down and interpreted the formulas for $d_1$, $d_2$, $N(d_1)$, and $N(d_2)$.
- Computed call and put prices with the implemented Python functions.
- Decomposed option prices into intrinsic value and time value.
- Verified put–call parity numerically.
- Applied the Black–Scholes formula to a sample option quote stored in the SQL database.

In the next notebook, we will focus on **implied volatility**:

- given a market price for an option,
- we will solve **numerically** for the volatility $\sigma$ that makes the Black–Scholes price match the market price.
